In [1]:
# Proyecto Final Sistemas de vision industrial
# Santiago Plá Rodríguez
# id:00397761



import serial
import time
import cv2
import numpy as np

# Configuración de conexión con Arduino
try:
    # Intentar establecer una conexión con Arduino en el puerto COM3
    arduino = serial.Serial('COM3', 4800, timeout=1)
    time.sleep(2)  # Esperar 2 segundos para que Arduino se inicialice
    print("Conexión establecida con Arduino")
except serial.SerialException:
    # Si ocurre un error al conectar, mostrar un mensaje
    print("Error al conectar con Arduino. Verifica el puerto.")
    arduino = None

# Variables globales
background = None  # Imagen del fondo para calibración
hand = None  # Objeto que almacena datos de la mano
frames_elapsed = 0  # Contador de fotogramas procesados
FRAME_HEIGHT = 300  # Altura del fotograma
FRAME_WIDTH = 400  # Anchura del fotograma
CALIBRATION_TIME = 80  # Tiempo para calibrar el fondo
BG_WEIGHT = 0.2  # Peso para el promedio del fondo
OBJ_THRESHOLD = 50  # Umbral para segmentar la mano
last_signal = None  # Última señal enviada a Arduino

# Región de interés (ROI) definida en la esquina superior derecha
region_top = 0
region_bottom = int(2 * FRAME_HEIGHT / 3)
region_left = int(FRAME_WIDTH / 2)
region_right = FRAME_WIDTH

# Clase para manejar datos de la mano
class HandData:
    """Clase que almacena datos relevantes de la mano como posición y estado."""
    def __init__(self, top, bottom, left, right, centerX):
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right
        self.centerX = centerX
        self.prevCenterX = 0
        self.isInFrame = False
        self.isWaving = False
        self.fingers = None
        self.gestureList = []

    def update(self, top, bottom, left, right):
        """Actualiza las posiciones de la mano."""
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right

    def check_for_waving(self, centerX):
        """Detecta si la mano está en movimiento horizontal."""
        self.prevCenterX = self.centerX
        self.centerX = centerX
        self.isWaving = abs(self.centerX - self.prevCenterX) > 3

# Función para enviar señal a Arduino
def send_signal_to_arduino(signal):
    """Envía una señal a Arduino si es diferente a la última señal enviada."""
    global last_signal
    if arduino and signal != last_signal:
        arduino.write(f"{signal}\n".encode('utf-8'))
        last_signal = signal
        print(f"Señal enviada a Arduino: {signal}")

# Función para mostrar en la ventana de la cámara
def write_on_image(frame):
    """Muestra texto en la imagen según el estado actual."""
    global hand
    text = "Buscando..."
    signal = None
    if frames_elapsed < CALIBRATION_TIME:
        text = "Calibrando..."
    elif hand is None or not hand.isInFrame:
        text = "Mano no detectada"
        signal = 3
    else:
        if hand.isWaving:
            text = "Moviendo"
            signal = 4
        elif hand.fingers == 0:
            text = "Cero"
            signal = 0
        elif hand.fingers == 1:
            text = "Uno"
            signal = 1
        elif hand.fingers == 2:
            text = "Dos"
            signal = 2

    if signal is not None:
        send_signal_to_arduino(signal)

    # Escribe el texto y dibuja el rectángulo de la ROI
    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.rectangle(frame, (region_left, region_top), (region_right, region_bottom), (255, 255, 255), 2)
    

# Funciones auxiliares
def get_region(frame):
    """Obtiene y prepara la región de interés del fotograma."""
    region = frame[region_top:region_bottom, region_left:region_right]
    region = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    region = cv2.GaussianBlur(region, (5, 5), 0)
    return region

def get_average(region):
    """Promedia los valores de la región para calibrar el fondo."""
    global background
    if background is None:
        background = region.copy().astype("float")
    else:
        cv2.accumulateWeighted(region, background, BG_WEIGHT)

def segment(region):
    """Segmenta la mano de la región de interés."""
    global background
    diff = cv2.absdiff(background.astype(np.uint8), region)
    thresholded_region = cv2.threshold(diff, OBJ_THRESHOLD, 255, cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(thresholded_region.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        if hand is not None:
            hand.isInFrame = False
        return None
    else:
        if hand is not None:
            hand.isInFrame = True
        segmented_region = max(contours, key=cv2.contourArea)
        return thresholded_region, segmented_region

def get_hand_data(thresholded_image, segmented_image):
    """Calcula posiciones y detecta dedos en la mano."""
    global hand
    convexHull = cv2.convexHull(segmented_image)
    top = tuple(convexHull[convexHull[:, :, 1].argmin()][0])
    bottom = tuple(convexHull[convexHull[:, :, 1].argmax()][0])
    left = tuple(convexHull[convexHull[:, :, 0].argmin()][0])
    right = tuple(convexHull[convexHull[:, :, 0].argmax()][0])
    centerX = int((left[0] + right[0]) / 2)

    if hand is None:
        hand = HandData(top, bottom, left, right, centerX)
    else:
        hand.update(top, bottom, left, right)

    if frames_elapsed % 6 == 0:
        hand.check_for_waving(centerX)

    hand.gestureList.append(count_fingers(thresholded_image))
    if frames_elapsed % 12 == 0:
        hand.fingers = most_frequent(hand.gestureList)
        hand.gestureList.clear()

def count_fingers(thresholded_image):
    """Cuenta la cantidad de dedos levantados."""
    global hand
    line_height = int(hand.top[1] + (0.2 * (hand.bottom[1] - hand.top[1])))
    line = np.zeros(thresholded_image.shape[:2], dtype=np.uint8)
    cv2.line(line, (thresholded_image.shape[1] - 1, line_height), (0, line_height), 255, 1)
    line = cv2.bitwise_and(thresholded_image, thresholded_image, mask=line)
    contours, _ = cv2.findContours(line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    fingers = 0
    for curr in contours:
        width = len(curr)
        if 5 < width < 3 * abs(hand.right[0] - hand.left[0]) / 4:
            fingers += 1
    return fingers

def most_frequent(input_list):
    """Encuentra el valor más frecuente en una lista."""
    counts = {}
    for item in input_list:
        counts[item] = counts.get(item, 0) + 1
    return max(counts, key=counts.get)

# Bucle principal
capture = cv2.VideoCapture(0)
while True:
    ret, frame = capture.read()
    frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
    frame = cv2.flip(frame, 1)
    region = get_region(frame)

    if frames_elapsed < CALIBRATION_TIME:
        get_average(region)
    else:
        region_pair = segment(region)
        if region_pair is not None:
            thresholded_region, segmented_region = region_pair
            cv2.drawContours(region, [segmented_region], -1, (255, 255, 255))
            cv2.imshow("Segmented Image", thresholded_region)
            get_hand_data(thresholded_region, segmented_region)

    write_on_image(frame)
    cv2.imshow("Camera Input", frame)
    frames_elapsed += 1

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

capture.release()
cv2.destroyAllWindows()


Conexión establecida con Arduino
Señal enviada a Arduino: 3
Señal enviada a Arduino: 0
Señal enviada a Arduino: 4
Señal enviada a Arduino: 1
Señal enviada a Arduino: 4
Señal enviada a Arduino: 2
Señal enviada a Arduino: 4
Señal enviada a Arduino: 1
Señal enviada a Arduino: 4
Señal enviada a Arduino: 1
Señal enviada a Arduino: 0
Señal enviada a Arduino: 4
Señal enviada a Arduino: 0
Señal enviada a Arduino: 4
Señal enviada a Arduino: 2
Señal enviada a Arduino: 1
Señal enviada a Arduino: 4
Señal enviada a Arduino: 0
Señal enviada a Arduino: 3
Señal enviada a Arduino: 0
Señal enviada a Arduino: 3
Señal enviada a Arduino: 0
Señal enviada a Arduino: 3
Señal enviada a Arduino: 0
Señal enviada a Arduino: 4
Señal enviada a Arduino: 3
Señal enviada a Arduino: 4
Señal enviada a Arduino: 3
Señal enviada a Arduino: 4
Señal enviada a Arduino: 3
Señal enviada a Arduino: 4
Señal enviada a Arduino: 0
Señal enviada a Arduino: 4
Señal enviada a Arduino: 0
Señal enviada a Arduino: 4
Señal enviada a Arduin